In [1]:
import pandas as pd
import feather

from keras.models import Sequential
from keras.layers import Dense
from keras.utils.np_utils import to_categorical
from keras.models import load_model


Using TensorFlow backend.


In [2]:
n_groups_per_col = 10
architecture = "2_dense_512"
path = "../data/models/dep/categorical/" + architecture +"/"

data_train = pd.read_feather(path + "data_train.feather").to_numpy()
labels_train = pd.read_feather(path + "labels_train.feather").to_numpy()

In [ ]:
def fit_ind_categorical_model(data_train, labels_train):
    
    model = Sequential()
    model.add(Dense(units=512, activation='relu', input_shape=(data_train.shape[1],)))
    model.add(Dense(units=512, activation='relu'))
    model.add(Dense(pow(n_groups_per_col, 3), activation='softmax'))
    
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    history = model.fit(data_train, labels_train, validation_split=0.2, batch_size=512, epochs=5, verbose=1)
    
    return (history, model)

In [ ]:
(history, model) = fit_ind_categorical_model(data_train, labels_train)
feather.write_dataframe(pd.DataFrame.from_dict(history.history), path + "history.feather", compression="uncompressed")
model.save(path + "model.hdf5")

In [3]:
### Predict
# model = load_model(path + "model.hdf5")


In [4]:
data_test = pd.read_feather(path + "data_test.feather").to_numpy()

#pred_prob_1 = model.predict_proba(data_test[0:500000,], batch_size = 512)
#feather.write_dataframe(pd.DataFrame(pred_prob_1), path + "pred_prob_1.feather", compression='uncompressed')

pred_prob_2 = model.predict_proba(data_test[500001:1000000,], batch_size = 512)
feather.write_dataframe(pd.DataFrame(pred_prob_2), path + "pred_prob_2.feather", compression='uncompressed')

#pred_prob_3 = model.predict_proba(data_test[1000001:,], batch_size = 512)
#feather.write_dataframe(pd.DataFrame(pred_prob_3), path + "pred_prob_3.feather", compression='uncompressed')
